# 🎙️ FlowTTS 语音合成 Demo

基于腾讯云 FlowTTS 的文字转语音服务。

**BYOK (Bring Your Own Key)** - 需要自带腾讯云凭证。

## 获取凭证

1. [腾讯云 API 密钥](https://console.cloud.tencent.com/cam/capi) - 获取 SecretId 和 SecretKey
2. [TRTC 控制台](https://console.cloud.tencent.com/trtc) - 获取 SdkAppId

## 相关链接

- [GitHub](https://github.com/chicogong/flowtts-byok-replicate)
- [Replicate](https://replicate.com/chicogong/flow-tts)
- [Hugging Face](https://huggingface.co/spaces/gonghaoran/flow-tts)

## 1. 安装依赖

In [ ]:
!pip install -q tencentcloud-sdk-python

## 2. 配置凭证

⚠️ **安全提示：** 请勿在公开的 Notebook 中保存真实凭证！

In [ ]:
# @title 输入腾讯云凭证 { display-mode: "form" }
SECRET_ID = "" # @param {type:"string"}
SECRET_KEY = "" # @param {type:"string"}
SDK_APP_ID = 0 # @param {type:"integer"}

# 验证凭证
if not SECRET_ID or not SECRET_KEY or not SDK_APP_ID:
    print("⚠️ 请填写完整的腾讯云凭证！")
else:
    print("✅ 凭证已配置")

## 3. 定义合成函数

In [ ]:
import io
import json
import wave
import base64
from IPython.display import Audio, display
from tencentcloud.common import credential
from tencentcloud.common.profile.client_profile import ClientProfile
from tencentcloud.common.profile.http_profile import HttpProfile
from tencentcloud.trtc.v20190722 import trtc_client, models

# Constants
MODEL = "flow_01_turbo"
ENDPOINT = "trtc.ai.tencentcloudapi.com"
REGION = "ap-beijing"


def pcm_to_wav(pcm_data: bytes, sample_rate: int = 24000) -> bytes:
    """Convert PCM to WAV format."""
    wav_buffer = io.BytesIO()
    with wave.open(wav_buffer, "wb") as wav_file:
        wav_file.setnchannels(1)
        wav_file.setsampwidth(2)
        wav_file.setframerate(sample_rate)
        wav_file.writeframes(pcm_data)
    return wav_buffer.getvalue()


def synthesize(
    text: str,
    voice_id: str = "v-female-R2s4N9qJ",
    speed: float = 1.0,
    volume: float = 1.0,
    language: str = "zh",
    sample_rate: int = 24000,
) -> bytes:
    """Synthesize speech from text using Tencent Cloud FlowTTS."""
    
    # Create client
    cred = credential.Credential(SECRET_ID, SECRET_KEY)
    http_profile = HttpProfile()
    http_profile.endpoint = ENDPOINT
    http_profile.reqTimeout = 120
    
    client_profile = ClientProfile()
    client_profile.httpProfile = http_profile
    
    client = trtc_client.TrtcClient(cred, REGION, client_profile)
    
    # Build request
    req = models.TextToSpeechSSERequest()
    params = {
        "Model": MODEL,
        "Text": text.strip(),
        "Voice": {
            "VoiceId": voice_id,
            "Speed": speed,
            "Volume": volume,
            "Language": language,
        },
        "AudioFormat": {
            "Format": "pcm",
            "SampleRate": sample_rate,
        },
        "SdkAppId": SDK_APP_ID,
    }
    req.from_json_string(json.dumps(params))
    
    # Call API and collect audio
    audio_chunks = []
    resp = client.TextToSpeechSSE(req)
    for event in resp:
        if isinstance(event, dict) and "data" in event:
            try:
                data = json.loads(event["data"].strip())
                if data.get("Type") == "audio" and data.get("Audio"):
                    audio_chunks.append(base64.b64decode(data["Audio"]))
                if data.get("IsEnd"):
                    break
            except (json.JSONDecodeError, KeyError):
                continue
    
    if not audio_chunks:
        raise ValueError("未收到音频数据")
    
    # Convert to WAV
    pcm_data = b"".join(audio_chunks)
    return pcm_to_wav(pcm_data, sample_rate=sample_rate)


print("✅ 函数定义完成")

## 4. 合成语音

修改下面的文本，然后运行单元格生成语音。

In [ ]:
# @title 语音合成参数 { display-mode: "form" }
text = "你好，欢迎使用 FlowTTS 语音合成服务。这是一个基于腾讯云的文字转语音演示。" # @param {type:"string"}
voice_id = "v-female-R2s4N9qJ" # @param {type:"string"}
speed = 1.0 # @param {type:"slider", min:0.5, max:2.0, step:0.1}
volume = 1.0 # @param {type:"slider", min:0.0, max:10.0, step:0.5}
language = "zh" # @param ["zh", "en", "yue", "ja", "ko", "auto"]
sample_rate = 24000 # @param [16000, 24000]

try:
    print(f"🎙️ 正在合成: {text[:50]}..." if len(text) > 50 else f"🎙️ 正在合成: {text}")
    wav_data = synthesize(
        text=text,
        voice_id=voice_id,
        speed=speed,
        volume=volume,
        language=language,
        sample_rate=int(sample_rate),
    )
    print("✅ 合成成功！")
    display(Audio(wav_data, rate=int(sample_rate)))
    
    # Save to file
    with open("output.wav", "wb") as f:
        f.write(wav_data)
    print("💾 已保存到 output.wav")
    
except Exception as e:
    print(f"❌ 合成失败: {e}")

## 5. 批量合成（可选）

如果你需要批量合成多段文本，可以使用下面的代码。

In [ ]:
# 批量合成示例
texts = [
    "第一段文本。",
    "第二段文本。",
    "第三段文本。",
]

for i, text in enumerate(texts):
    try:
        print(f"[{i+1}/{len(texts)}] 合成中: {text}")
        wav_data = synthesize(text)
        with open(f"output_{i+1}.wav", "wb") as f:
            f.write(wav_data)
        print(f"  ✅ 已保存到 output_{i+1}.wav")
    except Exception as e:
        print(f"  ❌ 失败: {e}")

print("\n🎉 批量合成完成！")